In [1]:
### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###
### For an AE which was the epoch that it achieved the lowest training loss? -best                  ###
### For the collection of the AEs, when it achieved the lowest of the worst performing ae? -max_min ###
### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###

In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import pickle_data

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch


from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.data_sets.shape_net import pc_loader as sn_pc_loader
from tf_lab.nips.helper import center_pclouds_in_unit_sphere
from tf_lab.icml.ae_farm_helper import EMD_Stats, Chamfer_Stats

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_name = 'chair'
ae_loss = 'emd'
n_pc_samples = 2048
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

In [5]:
# Load Raw Point-Clouds of class
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
syn_id = shape_net_category_to_synth_id()[class_name]
pclouds_path = osp.join(pclouds_path, syn_id)
file_names = pio.load_filenames_of_input_data(pclouds_path, '.ply')
pclouds, model_ids, syn_ids = pio.load_crude_point_clouds(file_names=file_names, n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(pclouds), )
train_data = PointCloudDataSet(pclouds)

6778 files containing complete point clouds were found.


In [6]:
# CONVOLUTIONAL ARCHS
if ae_loss == 'emd':
    experiment_stats = EMD_Stats
else:
    experiment_stats = Chamfer_Stats

ae_ids = experiment_stats.experiments_ids

losses = dict()
for ae_id in ae_ids:
    ae_name = '_'.join(['ae_farm', class_name, 'conv_arch', str(ae_id), str(n_pc_samples) + 'pts', ae_loss])
    train_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_name)
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    bneck = conf.encoder_args['n_filters'][-1]
    print bneck 
    assert(experiment_stats.experiment_id_to_bneck[ae_id] == bneck)
    saved_epochs = read_saved_epochs(conf.train_dir)
    reset_tf_graph()
    ae = PointNetAutoEncoder(ae_name, conf)    
    print ae_name
    losses[ae_name] = []
    for epoch in saved_epochs:
        ae.restore_model(conf.train_dir, epoch, verbose=True)
        loss_at_epoch = ae.evaluate(train_data, conf)[1]
        losses[ae_name].append((epoch, loss_at_epoch))

4
ae_farm_chair_conv_arch_18_2048pts_emd
Model restored in epoch 910.
Model restored in epoch 920.
Model restored in epoch 930.
Model restored in epoch 940.
Model restored in epoch 950.
Model restored in epoch 960.
Model restored in epoch 970.
Model restored in epoch 980.
Model restored in epoch 990.
Model restored in epoch 1000.


In [6]:
# MLP ARCHS
losses = dict()
ae_id = 6
ae_name = '_'.join(['ae_farm', class_name, 'mlp_arch', str(ae_id), str(n_pc_samples) + 'pts', ae_loss])
train_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_name)
conf = Conf.load(osp.join(train_dir, 'configuration'))
bneck = conf.encoder_args['n_filters'][-1]
print bneck 

saved_epochs = read_saved_epochs(conf.train_dir)
reset_tf_graph()
ae = PointNetAutoEncoder(ae_name, conf)    

losses[ae_name] = []

for epoch in saved_epochs[-10:]:
    ae.restore_model(conf.train_dir, epoch, verbose=True)
    loss_at_epoch = ae.evaluate(train_data, conf)[1]
    losses[ae_name].append((epoch, loss_at_epoch))

4
Model restored in epoch 910.
Model restored in epoch 920.
Model restored in epoch 930.
Model restored in epoch 940.
Model restored in epoch 950.
Model restored in epoch 960.
Model restored in epoch 970.
Model restored in epoch 980.
Model restored in epoch 990.
Model restored in epoch 1000.


In [7]:
epochs_ = [l[0] for l in losses[ae_name]]
losses_ = [l[1] for l in losses[ae_name]]
repo = np.argmin(losses_)
print losses_[repo], epochs_[repo]

108.468496492 990


In [16]:
losses_

[64.51379999885097,
 64.359766247913385,
 64.501950304217658,
 64.561083617200538,
 64.202133845072922,
 64.381325347756317,
 64.16416609677708,
 64.375140158888797,
 63.921085462347882,
 64.227192599290092]

In [24]:
prefix = 'ae_farm_chair_conv_arch_'
postfix = '_2048pts_' + ae_loss
ae_tag_to_id = lambda ae_name: int(ae_name[len(prefix):-len(postfix)])

In [29]:
import operator
for ae_example in losses.keys():
    ae_id = ae_tag_to_id(ae_example)
    bneck = experiment_stats.experiment_id_to_bneck[ae_id]
    losses[ae_example].sort(key=operator.itemgetter(1))
    print bneck, losses[ae_example][0]

512 (990, 50.992049003639345)
64 (940, 54.250119469024398)
128 (990, 51.128213850366365)
32 (960, 56.104974105668369)
256 (980, 51.021433926219608)


In [34]:
worst_of_best_value = 56.104974105668369
for ae_example in losses.keys():
    ae_id = ae_tag_to_id(ae_example)
    bneck = experiment_stats.experiment_id_to_bneck[ae_id]
    l_ae_sorted = np.array([pair[1] for pair in losses[ae_example]])
    loc = np.searchsorted(l_ae_sorted, worst_of_best_value)
    epoch_ae_sorted = np.array([pair[0] for pair in losses[ae_example]])
    print bneck, epoch_ae_sorted[loc], l_ae_sorted[loc]

512 260 56.4351911322
64 590 56.1521346463
128 290 56.2572510551
32 960 56.1049741057
256 290 56.3530345713
